In [32]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K

import numpy as np

In [61]:
def data_generator():
    
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = x_train.astype(np.float32) / 255
    x_train = x_train.reshape(60000,28,28,1)
    
    return x_train

In [62]:
x= data_generator()
x.shape

(60000, 28, 28, 1)

In [63]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K

def sampling(args):
    """Reparameterization function by sampling from an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def build_vae(x_shape, z_dimension):
    inputs = layers.Input(shape=x_shape)

    y = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    y = layers.LeakyReLU()(y)
    y = layers.Conv2D(32, 3, strides=2, padding="same")(y)
    y = layers.LeakyReLU()(y)
    y = layers.Conv2D(32, 3, strides=2, padding="same")(y)
    y = layers.LeakyReLU()(y)
    #y = layers.Conv2D(32, 3, strides=2, padding="same")(y)
    #y = layers.LeakyReLU()(y)
    y = layers.Conv2D(32, 3, strides=2, padding="same")(y)
    y = layers.LeakyReLU()(y)
    y_shape = y.shape

    y = layers.Flatten()(y)
    y = layers.Dense(1024, activation='relu')(y)
    y = layers.Dense(1024, activation='relu')(y)

    z_mean = layers.Dense(z_dimension, name="z_mean")(y)
    z_log_var = layers.Dense(z_dimension, name="z_log_var")(y)
    z = layers.Lambda(sampling)([z_mean, z_log_var])

    decoder_input = layers.Input(shape=(z_dimension,))
    
    
    y = layers.Dense(1024, activation='relu')(decoder_input)
    y = layers.Dense(1024, activation='relu')(y)
    y = layers.Dense(y_shape[1]*y_shape[2]*y_shape[3], activation="relu")(y)
    y = layers.Reshape(y_shape[1:])(y)

    y = layers.Conv2DTranspose(32, 3, strides=2, padding="same")(y)
    y = layers.LeakyReLU()(y)
    y = layers.Conv2DTranspose(32, 3, strides=2, padding="same")(y)
    y = layers.LeakyReLU()(y)
    #y = layers.Conv2DTranspose(32, 3, strides=2, padding="same")(y)
    #y = layers.LeakyReLU()(y)
    y = layers.Conv2DTranspose(32, 3, strides=2, padding="same")(y)
    y = layers.LeakyReLU()(y)
    y = layers.Conv2DTranspose(x_shape[2], 3, strides=2, padding="same")(y)
    y = layers.Activation('sigmoid')(y)

    encoder = models.Model(inputs, [z_mean, z_log_var, z], name ='encoder')
    decoder = models.Model(decoder_input, y, name='decoder')
    outputs = decoder(encoder(inputs)[2])
    vae = models.Model(inputs, outputs, name = 'vae')

    return encoder, decoder, vae, z_log_var, z_mean

In [64]:
encoder, decoder, vae, z_log_var, z_mean = build_vae((28, 28,1), 10)

In [68]:
latent_loss = -0.5 * K.sum(
    1+z_log_var - K.exp(z_log_var) - K.square(z_mean),
    axis=-1)
vae.add_loss(K.mean(latent_loss) / 784.)
vae.compile(optimizer="rmsprop")

In [66]:
x_train.shape

(60000, 28, 28)

In [67]:
#compile & fit
x_train = data_generator()
history = vae.fit(x_train, x_train, epochs=50, batch_size=128)

ValueError: A target array with shape (60000, 28, 28, 1) was passed for an output of shape (None, 32, 32, 1) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.

In [49]:
# 결과 확인



